In [ ]:
%run sources/slicer.py
%run sources/time_series_to_intervals.py
%run sources/time_series_event_merge.py

main_value = "p02" #variable
main_feature = "participant"


In [ ]:
feature = "distance"
p = pd.read_csv(f"processed_data/fitbit/{feature}.csv")
p[p[main_feature] == main_value][feature].describe()

In [ ]:
time_feature = "vt"
"""
timeseries_df_list = {"paths": ["processed_data/fitbit/calories.csv",
                                "processed_data/fitbit/distance.csv",
                                "processed_data/fitbit/steps.csv",
                                "processed_data/fitbit/heart_rate.csv"],
                       "group_features": ["calories", "distance", "steps", "bpm"],
                       "bucket_values": [[1.2, 1.57], [1860], [26], [46, 53, 62]]} #variable
"""
timeseries_df_list = {"paths": [],
                       "group_features": [],
                       "bucket_values": []}

event_df_list = {"paths": ["processed_data/fitbit/exercise.csv",
                            "processed_data/fitbit/sleep_score.csv",
                            "processed_data/fitbit/sleep.csv"],
                  "group_features": ["Activity", "label", "sleep_phase"],
                  "value_feature": ["value", "value", "value"]}

slicer_params = {"stride": 24, "ow": 20, "ww": 4, "pw": 48, "granularity": "hours"} #variable

final_odf, final_pdf = samples_from_dataset(main_value, main_feature, time_feature, timeseries_df_list, event_df_list, slicer_params)

In [ ]:
final_odf[final_odf.i == 6]

In [ ]:
def sleep_score_from_prediction_window(dfs):
    r,ids =[],[]
    for df in dfs["label"]:
        scs = df.to_dict(orient="records")
        scs.sort(key=lambda x: x['vt'])
        if len(df) > 0:
            r.append(scs[0]['value']) 
            ids.append(scs[0]['i'])   
    return pd.DataFrame({'SleepScore':r, 'i':ids})




In [ ]:
def df_to_sequences(df):
    df["sequence"] = df.apply(lambda row: (row['vt'], row['label'], row['value']), axis=1)
    r =df.groupby("i")["sequence"].agg(list).reset_index()
    for i in range(len(r)):
        r.iloc[i]["sequence"].sort(key=lambda x: x[0])
    return r

Z = df_to_sequences(final_odf)

In [ ]:
Z = pd.merge(Z, sleep_score_from_prediction_window(final_pdf), on='i', how='inner')

In [ ]:
Z["SleepScore"].describe()

In [ ]:
def score_buckets(s, buckets):
    for i in range(len(buckets)):
        if s <= buckets[i]:
            if i == 0 or s > buckets[i-1]:
                return i 
    return len(buckets)    

scores = [67,73,76] #variable

Z["class"] = Z["SleepScore"].apply(lambda x: score_buckets(x, scores))

In [ ]:
Z

In [ ]:
import json
from datetime import datetime

In [ ]:
ZPARAMS = {
    "main_value":main_value, 
    "main_feature":main_feature, 
    "time_feature":time_feature, 
    "timeseries_df_list":timeseries_df_list, 
    "event_df_list":event_df_list, 
    "slicer_params":slicer_params,
    "class":{"SleepScore": scores}
}
TIME = datetime.now().strftime('%Y%m%d_%H%M')
ZNAME = f"{TIME}_fitbit_{main_value}_events"


In [ ]:
Z[["sequence", "class"]].to_csv(f"datasets/{ZNAME}.csv", index=False)
with open(f"datasets/{ZNAME}.json", 'w') as json_file:
    json.dump(ZPARAMS, json_file, indent=2)
